In [34]:
import pandas as pd
import numpy as np

## Descripción del problema a resolver

**La base de datos** que se utilizará para probar el paquete implementado por el equipo corresponde a una representación simplificada de la Red Eléctrica Mexicana, que se utiliza para realizar la planeación del sistema nacional de generación, transmisión y distribución de energía eléctrica.

La información a la que se tuvo acceso proviene del Centro Nacional de Control de Energía ([CENACE](https://www.gob.mx/cenace)) y su publicación se realiza de forma anual en los Programas de Ampliación y Modernización de la Red Nacional de Transmisión y Redes Generales de Distribución ([ver documento PAMRNT](https://www.cenace.gob.mx/Docs/10_PLANEACION/ProgramasAyM/Programa%20de%20Ampliaci%C3%B3n%20y%20Modernizaci%C3%B3n%20de%20la%20RNT%20y%20RGD%202021%20-%202035.pdf)).

En la Figura 4.3.2 de ese documento, se muestra la topología que tiene la red que representa las regiones o zonas más representativas (en cuanto a demanda y generación de energía eléctrica o bien por cuestiones de ubicación geográfica), así como su conectividad. Adicionalmente, cada uno de los arcos (ramas) tiene una capacidad definida de transmisión de energía, comunmente llamada _límite de transmisión entre regiones_.

<p align = "center">
    <img src="images/red_nacional.png" width="1329" height="911" />

        fuente: Elaborado por CENACE

El grafo que observamos es de tipo "no-dirigido", porque en una red eléctrica el sentido del flujo de potencia (energía) puede darse en cualquier sentido y está determinado por la solución que se obtenga del problema de [Flujos de Potencia](https://www.intechopen.com/chapters/65445). Sin embargo, para el ejercicio que realizaremos en esta práctica, partiremos de una suposición de sentido en los flujos de potencia basada en las condiciones que predominan en la red eléctrica y que se reportan en el PAMRNT (en la sección: _Condiciones operativas en las transferencias de potencia en los principales enlaces del Sistema Eléctrico Nacional en la demanda máxima de verano de 2020_).

**El Planteamiento** del problema que se pretende resolver es el siguiente: "Dada la red eléctrica de la Figura 4.3.2 encontrar flujo máximo que se puede transmitir en la red".

**¿Para qué puede servir encontrar el flujo máximo en la red eléctrica?**

- Para determinar los posibles cuellos de botella (restricciones) que se pueden presentar al tratar de enviar energía desde un punto de la red a otro.
- Encontrar posibles puntos de inyección donde resulte más conveniente instalar generación (que se obtengan mayores flujos máximos por la red)
- Descubrir cuales corredores de trasnmisión (rutas) se ven más utilizadas cuando la inyección de energía se presenta en algún punto de la red.


## Lectura y limpieza de la Base de Datos

In [51]:
red = pd.read_csv('BD/red.csv')
pd.options.display.max_rows = 10
red.shape

(2289, 5)

In [52]:
red.dtypes

Enlace     object
Periodo    object
Cap        object
Real       object
Img        object
dtype: object

In [54]:
red = red.applymap(lambda x: x.replace(',', ''))
red['Cap'] = red['Cap'].astype('float')

In [81]:
enlaces = red[['Enlace','Cap']]
suma = enlaces.groupby(['Enlace']).aggregate(np.sum)
suma['Enlace'] = suma.index
env_rec = suma['Enlace'].str.split('-', expand=True)
envio = env_rec[0].to_numpy()
recep = env_rec[1].to_numpy()

In [82]:
recep

array(['AGUA', 'LEON', 'SLP', 'CHIH', '6ABR', 'LCAR', 'SALM', 'POZA',
       'PUEB', 'BELI', 'CUAU', 'RIOE', 'MAZ', 'CENT', 'LCAR', 'AGUA',
       'CANC', 'PILV', 'PLYC', 'CHET', 'LERM', 'MERI', 'COAT', 'GUAT',
       'TABA', 'AGUA', 'CARA', 'MANZ', 'SALM', 'LCAR', 'HERM', 'HUA',
       '6ABR', 'CANA', 'NACO', 'POZ', 'JUIL', 'CENT', 'DOGU', 'JUIL',
       'MROP', 'OAXA', 'GRIJ', 'TEMA', 'DUR', 'SALT', '1ROM', 'CAMA',
       'ACAP', 'SALA', 'MERI', 'DUR', 'TEP', 'CHET', 'DZT', 'VALLA',
       'CUL', 'CHIH', 'JUA', 'NVOC', 'GUEZ', 'REY', 'RIO', 'SALT', 'ACAP',
       'PUEB', 'TOLU', 'COAT', 'JUIL', 'CANA', 'REY', 'NACO', 'ACAP',
       'MOCH', 'GUAY', 'CANC', 'PLYC', 'CANC', 'COZU', 'PUEB', 'TULA',
       'VERA', 'CENT', 'JILO', 'MAT', 'NLAD', 'QUER', '1ROM', 'SLPZ',
       'QUER', 'ESCA', 'HUA', 'JILO', 'QUER', 'COAT', 'OAXA', 'PUEB',
       'GUA', 'CENT', 'DOGU', 'CENT', 'JILO', 'QUER', 'HUA', 'SLP',
       'CANC', 'PLYC', 'POZ', 'TEMA'], dtype=object)